In [19]:
import cv2 as cv
import os 
import numpy as np
import time
import imgaug as ia
from imgaug import augmenters as iaa
import os
import sys
import argparse
from glob import glob
import cv2
import shutil
import random
from PIL import Image
import pandas as pd
import errno


In [20]:
def video_generator(path,person_name):  
    # Create an object to read 
    # from camera
    video = cv2.VideoCapture(0)

    # We need to check if camera
    # is opened previously or not
    if (video.isOpened() == False): 
        print("Error reading video file")

    # We need to set resolutions.
    # so, convert them from float to integer.
    frame_width = int(video.get(3))
    frame_height = int(video.get(4))

    size = (frame_width, frame_height)

    # Below VideoWriter object will create
    # a frame of above defined The output 
    # is stored in 'filename.avi' file.
    
    result = cv2.VideoWriter(path+'/'+person_name+'.avi', 
                             cv2.VideoWriter_fourcc(*'MJPG'),
                             10, size)
    capture_duration = 10
    start_time = time.time()
    while(int(time.time() - start_time) < capture_duration):
        ret, frame = video.read()

        if ret == True: 

            # Write the frame into the
            # file 'filename.avi'
            result.write(frame)

            # Display the frame
            # saved in the file
            cv2.imshow('Frame', frame)

            # Press S on keyboard 
            # to stop the process
            if cv2.waitKey(1) & 0xFF == ord('s'):
                break

        # Break the loop
        else:
            break
    video.release()
    result.release()

    # Closes all the frames
    cv2.destroyAllWindows()

    print("The video was successfully saved")

In [21]:
def image_generator(path,person_name):
    cam = cv2.VideoCapture(path+'/'+person_name+'.avi')
    currentframe = 0
    i=0
    while(True):
        ret,frame = cam.read()
        
        if ret:
            name = path+'/'+'image(' + str(currentframe) + ').jpg'
            
            cv2.imwrite(name, frame)
            currentframe += 1
            i+=1
        if i==100:
            break
        else:
            break
    os.remove(path+'/'+person_name+'.avi')
    cam.release()
    cv2.destroyAllWindows()
    print(f'{i}:Image are Generated')

In [22]:
def data_agumentation(path,n):

    IMG_DIR=path
    if not os.path.isdir(IMG_DIR):
        print('%s is not a valid directory.' % IMG_DIR)
        sys.exit(1)
    IMG_EXTENSION = '.jpg'
    NUM_AUG_IMAGES = n
    cwd = os.getcwd()

    #### Define augmentation sequence ####
    # This can be tweaked to create a huge variety of image augmentations.
    # See https://github.com/aleju/imgaug for a list of augmentation techniques available.
    seq1 = iaa.Sequential([
        iaa.Fliplr(0.5),                             # Horizontal flip 50% of images
        iaa.Crop(percent=(0, 0.10)),                 # Crop all images between 0% to 10%
        iaa.GaussianBlur(sigma=(0, 1)),              # Add slight blur to images
    ##    iaa.Multiply((0.7, 1.3), per_channel=0.2),   # Slightly brighten, darken, or recolor images
    ##    iaa.Affine(
    ##        scale={"x": (0.8, 1.2), "y": (0.8,1.2)},                # Resize image
    ##        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # Translate image
    ##        rotate=(-5, 5),                                         # Rotate image
    ##        mode=ia.ALL, cval=(0,255)                               # Filling in extra pixels
    ##        )
        ])


    #### Start of main program ####

    # Obtain list of images in IMG_DIR directory
    img_fns = glob(IMG_DIR + '/*' + IMG_EXTENSION)

    # Go through every image in directory, augment it, and save new image/annotation data
    count=0
    for img_fn in img_fns:

        #---- Load image ----#
        img1_bgr = cv2.imread(img_fn) # Load image with OpenCV
        img1 = cv2.cvtColor(img1_bgr, cv2.COLOR_BGR2RGB) # Re-color to RGB from BGR

        #---- Augment image N times----#
        for i in range(NUM_AUG_IMAGES):
            img_aug1 = seq1(images=[img1])[0] # Apply augmentation to image

            #---- Save image ----#
            base_fn = img_fn.replace(IMG_EXTENSION,'') # Get image base filename
            img_aug_fn = base_fn + ('_aug%d' % (i+1)) + IMG_EXTENSION # Append "aug#" to filename
            img_aug_bgr1 = cv2.cvtColor(img_aug1, cv2.COLOR_RGB2BGR) # Re-color to BGR from RGB
            cv2.imwrite(img_aug_fn,img_aug_bgr1) # Save image to disk
            count+=1
    print(f'Image Agumentation successfully Done !! {count} images Agumented')

In [23]:
def cleanup(folder):
    for name in os.listdir(folder):
        try:
            os.rmdir(os.path.join(folder, name))
        except:
            continue
            


In [24]:
def image_rescaling(path):
    for img in os.listdir(path):
        img_dir=os.path.join(path,img)
        img = Image.open(img_dir)
        img = img.resize((28, 28)) 
        img.save(img_dir)
    print('image_rescaled')
        
        
        
    

In [25]:
perm=int(input('Enter Number of person for no :'))
person_list={}
person_dir_path={}
for i in range(1,perm+1):
    person=input('Enter person name:')
    person_list[person]=i
    person_data=os.path.join('data',person)
    person_dir_path[person]=person_data
    os.mkdir(person_data)
    print('Show _object')
    video_generator(person_data,person)
    image_generator(person_data,person)
    image_rescaling(person_data)
    data_agumentation(person_data,100)
    

Enter Number of person for no :5
Enter person name:enjoy_cup
Show _object
The video was successfully saved
1:Image are Generated
image_rescaled
Image Agumentation successfully Done !! 100 images Agumented
Enter person name:smile_cup
Show _object
The video was successfully saved
1:Image are Generated
image_rescaled
Image Agumentation successfully Done !! 100 images Agumented
Enter person name:medicine
Show _object
The video was successfully saved
1:Image are Generated
image_rescaled
Image Agumentation successfully Done !! 100 images Agumented
Enter person name:coffee
Show _object
The video was successfully saved
1:Image are Generated
image_rescaled
Image Agumentation successfully Done !! 100 images Agumented
Enter person name:ketchup
Show _object
The video was successfully saved
1:Image are Generated
image_rescaled
Image Agumentation successfully Done !! 100 images Agumented


In [26]:
def ratio_train(n):
    return int((80/100)*n)
def ratio_val(n):
    return int((20/100)*n)

# data_preprocessing

In [27]:
train_data=os.path.join('data','train')
os.mkdir(train_data)
val_data=os.path.join('data','val')
os.mkdir(val_data)

In [28]:

for key,path in person_dir_path.items():
    class_train_path=os.path.join(train_data,key)
    os.mkdir(class_train_path)
    for i,img in enumerate(os.listdir(path)):
        source=os.path.join(path,img)
        shutil.move(source,class_train_path)
    class_val_path=os.path.join(val_data,key)
    os.mkdir(class_val_path)
    for img in random.sample(os.listdir(class_train_path),ratio_val(len(os.listdir(class_train_path)))):
        source=os.path.join(class_train_path,img)
        shutil.move(source,class_val_path)

In [29]:
#labels _formation
person_list

{'enjoy_cup': 1, 'smile_cup': 2, 'medicine': 3, 'coffee': 4, 'ketchup': 5}

In [30]:
labels={}
for key,path in person_dir_path.items():
    dir_paths=os.path.join(train_data,key)
    labels[key]=np.full(len(os.listdir(dir_paths)),person_list[key])


{'enjoy_cup': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'smile_cup': array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 'medicine': array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
 'coffee': array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [32]:
dir_class_img={}
for key,path in person_dir_path.items():
    dir_paths=os.path.join(train_data,key)
    img_dict={}
    for i,img in enumerate(os.listdir(dir_paths)):
        img_pi = Image.open(os.path.join(dir_paths,img))
        arr_img=np.asarray(img_pi)
        img_dict[i]=arr_img.flatten()
    dir_class_img[key]=img_dict

In [34]:
data=pd.DataFrame()
lab_data=pd.DataFrame()
for (key1,values),(lab_key,lab_values) in zip(dir_class_img.items(),labels.items()):
    temp_df=pd.DataFrame(values)
    temp_df=temp_df.T
    data=pd.concat([data,temp_df],axis=0)
    lab_data_temp=pd.DataFrame(lab_values)
    lab_data=pd.concat([lab_data,lab_data_temp],axis=0)  

In [35]:
final_data_set=pd.concat([data,lab_data],axis=1)

In [37]:
final_data_set = final_data_set.apply(np.random.permutation, axis=0)    

In [57]:
final_data_set.to_csv('data.csv',index=False)

In [62]:
label_data=pd.DataFrame(labels)
label_data.to_csv('classes.csv',index=False)